In [1]:
import numpy as np
import seaborn as sns
import torch

import umap
import matplotlib.pyplot as plt
import pandas as pd
from community import community_louvain
from torch_geometric.utils import k_hop_subgraph,to_networkx,from_networkx
import matplotlib

import utils
import plots
from model_AE import reduction_AE
from model_GAT import Encoder,SenGAE,train_GAT
from model_Sencell import Sencell

import logging

logging.basicConfig(format='%(asctime)s.%(msecs)03d [%(levelname)s] [%(filename)s:%(lineno)d] %(message)s',
                    datefmt='# %Y-%m-%d %H:%M:%S')

logging.getLogger().setLevel(logging.DEBUG)
logger = logging.getLogger()

# Part 1: load and process data
data_path="/users/PCON0022/haocheng/Basu_lab/rmarkdown/SCB01S5.h5ad"
# cell_cluster_arr在画umap的时候用
adata,cluster_cell_ls,cell_cluster_arr,celltype_names=utils.load_data(data_path)
# plots.umapPlot(adata.obsm['X_umap'],clusters=cell_cluster_arr,labels=celltype_names)

new_data,markers_index,\
sen_gene_ls,nonsen_gene_ls,gene_names=utils.process_data(adata,cluster_cell_ls,cell_cluster_arr)

print(f'cell num: {new_data.shape[0]}, gene num: {new_data.shape[1]}')

gene_cell=new_data.X.toarray().T
graph_nx=utils.build_graph_nx(gene_cell,cell_cluster_arr,sen_gene_ls,nonsen_gene_ls,gene_names)
logger.info("Part 1, data loading and processing end!")

# Part 2: generate init embedding
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device: ',device)

retrain=False
if retrain:
    gene_embed,cell_embed=reduction_AE(gene_cell,device)
    logger.info(gene_embed.shape,cell_embed.shape)
    torch.save(gene_embed,'./gene.emb')
    torch.save(cell_embed,'./cell.emb')
else:
    gene_embed=torch.load('./gene.emb')
    cell_embed=torch.load('./cell.emb')

graph_nx=utils.add_nx_embedding(graph_nx,gene_embed,cell_embed)
graph_pyg=utils.build_graph_pyg(gene_cell,gene_embed,cell_embed)
logger.info("Part 2, AE end!")

# Part 3: train GAT
# graph_pyg=graph_pyg.to('cpu')
GAT_model=train_GAT(graph_nx,graph_pyg,device,retrain=False,resampling=False)
logger.info("Part 3, training GAT end!")



from sampling import identify_sencell_marker_graph
from sampling import sub_sampling_by_random
from model_Sencell import cell_optim,update_cell_embeddings,old_cell_optim

from sampling import identify_sengene_then_sencell


all_gene_ls=[]

list_sencell_cover=[]
list_sengene_cover=[]

def get_sencell_cover(old_sencell_dict,sencell_dict):
    set1=set(list(old_sencell_dict.keys()))
    set2=set(list(sencell_dict.keys()))
    set3=set1.intersection(set2)
    print('sencell cover:',len(set3)/len(set2))
    
    return len(set3)/len(set2)

def get_sengene_cover(old_sengene_ls,sengene_ls):
    set1=set(old_sengene_ls)
    set2=set(sengene_ls)
    set3=set1.intersection(set2)
    print('sengene cover:',len(set3)/len(set2))
    
    return len(set3)/len(set2)
    
    
cellmodel=Sencell().to(device)
optimizer = torch.optim.Adam(cellmodel.parameters(), lr=0.001, 
                                weight_decay=1e-3)



# 2022-11-22 12:33:17.390 [DEBUG] [attrs.py:77] Creating converter from 3 to 5


cluster 数量： 21
celltype names: ['Macrophages', 'T cell lineage', 'Unknown', 'B cell lineage', 'Innate lymphoid cell NK', 'AT2', 'Monocytes', 'Multiciliated lineage', 'Dendritic cells', 'EC capillary', 'Mast cells', 'Fibroblasts', 'Secretory', 'EC venous', 'Lymphatic EC mature', 'AT1', 'Basal', 'EC arterial', 'Myofibroblasts', 'None', 'Submucosal Secretory']
-----------------------  ----
Macrophages              6941
T cell lineage            749
Unknown                   618
B cell lineage            374
Innate lymphoid cell NK   327
AT2                       294
Monocytes                 228
Multiciliated lineage     194
Dendritic cells           177
EC capillary              138
Mast cells                100
Fibroblasts                93
Secretory                  86
EC venous                  74
Lymphatic EC mature        68
AT1                        27
Basal                      26
EC arterial                20
Myofibroblasts             17
None                        6
Submucosal

# 2022-11-22 12:33:23.519 [INFO] [3655344800.py:39] Part 1, data loading and processing end!


device:  cuda:0
the number of edges: 1745053
edge index:  torch.Size([2, 1745053])
node feature:  torch.Size([12803, 128])
Pyg graph: Data(x=[12803, 128], edge_index=[2, 3490106], y=[12803])


# 2022-11-22 12:33:27.310 [INFO] [3655344800.py:57] Part 2, AE end!
# 2022-11-22 12:33:27.319 [INFO] [3655344800.py:62] Part 3, training GAT end!


graph.is_directed(): False


In [ ]:
all_marker_index=sen_gene_ls
    
iteration_results=[]
for iteration in range(5):
    logger.info(f"iteration: {iteration}")
    sampled_graph,sencell_dict,nonsencell_dict,cell_clusters,big_graph_index_dict=sub_sampling_by_random(graph_nx,
                                                            sen_gene_ls,
                                                            nonsen_gene_ls,
                                                            GAT_model,
                                                            all_marker_index,
                                                            n_gene=len(all_marker_index),                                                        
                                                            gene_num=2245,cell_num=10558,
                                                            gene_rate=0.3,cell_rate=0.5,
                                                            debug=False)
    old_sengene_indexs=all_marker_index
    for epoch in range(10):
        logger.info(f"epoch: {epoch}")
        old_sencell_dict=sencell_dict
        cellmodel,sencell_dict,nonsencell_dict=cell_optim(cellmodel,optimizer,
                                                          sencell_dict,nonsencell_dict,device,
                                                         train=True)
        sampled_graph=update_cell_embeddings(sampled_graph,sencell_dict,nonsencell_dict)
        sencell_dict,nonsencell_dict, \
        sen_gene_indexs,nonsen_gene_indexs=identify_sengene_then_sencell(sampled_graph,GAT_model,
                                                                      sencell_dict,nonsencell_dict,
                                                                      cell_clusters,
                                                                      big_graph_index_dict,
                                                                      len(all_marker_index))

        get_sencell_cover(old_sencell_dict,sencell_dict)
        get_sengene_cover(old_sengene_indexs,sen_gene_indexs)
        old_sengene_indexs=sen_gene_indexs
    iteration_results.append([sen_gene_indexs,sencell_dict])

# 2022-11-22 12:33:27.331 [INFO] [2913966771.py:5] iteration: 0


Start sampling subgraph randomly ...
    Sengene num: 351, Nonsengen num: 351
subgraph total node num: (11260,)
After sampling, gene num:  tensor(702)


# 2022-11-22 12:35:21.368 [INFO] [2913966771.py:17] epoch: 0


    Sencell num: 321, Nonsencell num: 3210
这一簇没有非老化细胞： 20
72.55316162109375
这一簇没有非老化细胞： 20
67.7896499633789
这一簇没有非老化细胞： 20
60.401214599609375
这一簇没有非老化细胞： 20
49.614994049072266
这一簇没有非老化细胞： 20
41.79710006713867
这一簇没有非老化细胞： 20
49.069705963134766
这一簇没有非老化细胞： 20
51.16868209838867
这一簇没有非老化细胞： 20
46.07640838623047
这一簇没有非老化细胞： 20
41.43885803222656
这一簇没有非老化细胞： 20
41.812034606933594
这一簇没有非老化细胞： 20
43.59840774536133
这一簇没有非老化细胞： 20
44.84477615356445
这一簇没有非老化细胞： 20
44.68792724609375
这一簇没有非老化细胞： 20
43.306724548339844
这一簇没有非老化细胞： 20
41.63454818725586
这一簇没有非老化细胞： 20
40.03664016723633
这一簇没有非老化细胞： 20
39.082427978515625
这一簇没有非老化细胞： 20
39.80221939086914
这一簇没有非老化细胞： 20
40.30607604980469
这一簇没有非老化细胞： 20
39.654266357421875
rechoice sengene num: 351 rechoice nonsengene num: 351


# 2022-11-22 12:42:25.282 [INFO] [2913966771.py:17] epoch: 1


...

    Sencell num: 599, Nonsencell num: 5990
sencell cover: 1.0
sengene cover: 0.9971509971509972
15.155023574829102
15.041267395019531
14.88664722442627
14.729249954223633
14.612979888916016
14.56500244140625
14.58199405670166
14.636119842529297
14.688108444213867
14.70687484741211
14.681159019470215
14.620616912841797
14.548260688781738
14.48780632019043
14.453946113586426
14.447403907775879
14.457419395446777
14.47061824798584
14.475290298461914
14.465812683105469
rechoice sengene num: 351 rechoice nonsengene num: 351


# 2022-11-22 16:29:01.937 [INFO] [2913966771.py:5] iteration: 3


    Sencell num: 600, Nonsencell num: 6000
sencell cover: 0.9983333333333333
sengene cover: 0.9971509971509972
Start sampling subgraph randomly ...
    Sengene num: 351, Nonsengen num: 351
subgraph total node num: (11260,)
After sampling, gene num:  tensor(702)


# 2022-11-22 16:31:07.418 [INFO] [2913966771.py:17] epoch: 0


    Sencell num: 329, Nonsencell num: 3290
这一簇没有非老化细胞： 20
703.7671508789062
这一簇没有非老化细胞： 20
477.7999267578125
这一簇没有非老化细胞： 20
237.04759216308594
这一簇没有非老化细胞： 20
86.91600036621094
这一簇没有非老化细胞： 20
44.630062103271484
这一簇没有非老化细胞： 20
41.27510070800781
这一簇没有非老化细胞： 20
34.865291595458984
